In [1]:
#get rates for H(5) + CHFCF2(700) <=> HF(684) + CF2CH(718)

import cantera as ct
import os 
import re
from rmgpy.molecule import Molecule
from rmgpy.reaction import Reaction
from rmgpy.species import Species
from rmgpy.data.kinetics.family import KineticsFamily
from rmgpy.data.rmg import RMGDatabase
from rmgpy import settings
from rmgpy.data.base import Database
from rmgpy.chemkin import load_species_dictionary
import rmgpy.chemkin
import math


In [90]:
#current model
full_path = '/work/westgroup/nora/Code/'

curr_RMG_chem = full_path + 'projects/PFAS/ESSCI/models/RMG/H_F_families_only/chemkin/chem0119.inp'
curr_RMG_dict = full_path + 'projects/PFAS/ESSCI/models/RMG/H_F_families_only/chemkin/species_dictionary.txt'
curr_RMG_trans = full_path + 'projects/PFAS/ESSCI/models/RMG/H_F_families_only/chemkin/tran.dat'

curr_species_list, curr_reaction_list = rmgpy.chemkin.load_chemkin_file(curr_RMG_chem, dictionary_path=curr_RMG_dict, transport_path=curr_RMG_trans)

In [54]:
#previous model
full_path = '/work/westgroup/nora/Code/'

prev_RMG_chem = full_path + 'projects/PFAS/models/C3H8_CH2F2/C3H8_CH2F2/second_try_on_rebase/chemkin/copies/copy_98_chem_annotated.inp'
prev_RMG_dict = full_path + 'projects/PFAS/models/C3H8_CH2F2/C3H8_CH2F2/second_try_on_rebase/chemkin/species_dictionary.txt'
prev_RMG_trans = full_path + 'projects/PFAS/models/C3H8_CH2F2/C3H8_CH2F2/second_try_on_rebase/chemkin/tran.dat'

prev_species_list, prev_reaction_list = rmgpy.chemkin.load_chemkin_file(prev_RMG_chem, dictionary_path=prev_RMG_dict, transport_path=prev_RMG_trans)

In [59]:
F_abs_rxns_prev = []
for rxn in prev_reaction_list:
    try: 
        if rxn.family=='F_Abstraction':
            F_abs_rxns_prev.append(rxn)
    except AttributeError as e: 
        continue
        
for rxn in F_abs_rxns_prev:
    print(rxn, '\t', rxn.get_rate_coefficient(1400,101325))

H(5) + CHFO(692) <=> HF(684) + HCO(14) 	 525.6200385331582
H(5) + CHFCF2(700) <=> HF(684) + CHFCF[Z](717) 	 170.82691764090723
H(5) + CHFCF2(700) <=> HF(684) + CF2CH(718) 	 205.64714132359074
CHFCF[Z](717) + CHFCF2(700) <=> CF2CH(718) + CHFCF2(700) 	 1.8712161791928972
CHFO(692) + CF2CH(718) <=> HCO(14) + CHFCF2(700) 	 1.3968163056956187
H(5) + CHF3(687) <=> HF(684) + CHF2(727) 	 60.909213141775304
CHF2(727) + CHFO(692) <=> CHF3(687) + HCO(14) 	 0.10793313231963368
CHF2(727) + CHFCF2(700) <=> CHF3(687) + CF2CH(718) 	 1.8398697905560226
CHF3(687) + CHFCF[Z](717) <=> CHF2(727) + CHFCF2(700) 	 2.2405167398732773
CHF2(727) + C2HF(703) <=> CHF3(687) + C2H(28) 	 0.15905828383274984


In [67]:
reaction_smiles = []
for rxn in F_abs_rxns_prev:
    reactant_smiles = []
    product_smiles = []
    for reactant in rxn.reactants: 
        reactant_smiles.append(reactant.smiles)
    for product in rxn.products: 
        product_smiles.append(product.smiles)
    reaction_smiles.append((reactant_smiles, product_smiles))

In [90]:
F_abs_rxns_prev[0]

In [91]:
F_abs_rxns_prev[0].get_rate_coefficient(1400, 101325)

525.6200385331582

In [3]:
A = 1.054700e-05
n = 5.457
Ea =26.866


get_rate_coefficient(A, n, Ea)

99464216.33432922

In [94]:
[rct.smiles for rct in F_abs_rxns_prev[0].reactants]


['[H]', 'O=CF']

In [2]:

T0=1
R = 8.314472
T=1400
def get_rate_coefficient(A, n, Ea):
    Ea_ = Ea*4184
    A_ = A/(10**6)
    rate = A_ * (T / T0) ** n * math.exp(-Ea_ / (R * T))
    rate_in_new_units = rate*(100**3)
    return rate_in_new_units


In [13]:
new = [
    {'A': 1.0547e-05, 'n': 5.457, 'Ea': 25.149}, # H(5) + CHFCF2(54) <=> HF(38) + CF2CH(72) 
    {'A': 6.87, 'n': 3.88, 'Ea': 29.732}, # H(5)+CHFO(49)<=>HF(36)+HCO(14)
    {'A': 3.6973e-04, 'n': 5.139, 'Ea': 30.586}, #  H(5) + CHFCF2(54) <=> HF(38) + CHFCF[Z](71)
    {'A': 5.861460e-04, 'n': 4.798, 'Ea': 40.973}, # CHFO(49) + CF2CH(740) <=> HCO(14) + CHFCF2(80)
    {'A': 1.920000e+00, 'n': 4.04, 'Ea': 32.935}, # H(5)+CHF3(717)<=>HF(36)+CHF2(25)
    {'A': 3.630000e+01, 'n': 3.37, 'Ea': 43.834}, # CHF2(25)+CHFO(49)<=>CHF3(717)+HCO(14)
    {'A': 1.527230e-01, 'n': 4.186, 'Ea': 39.636}, #CHF2(25) + CHFCF2(80) <=> CHF3(717) + CF2CH(740)
    {'A': 8.880000e+01, 'n': 3.23, 'Ea': 46.348} # CHF3(717) + CHFCF[Z](739) <=> CHF2(25) + CHFCF2(80)
]   

  
old = [
    {'A': 1.575380e-09, 'n':6.622, 'Ea': 23.817}, # H(5) + C2HF3(700) <=> HF(684) + CF2CH(718)
    {'A': 3.318290e-02, 'n': 4.482, 'Ea': 24.991}, # H(5) + CHFO(692) <=> HF(684) + HCO(14)
    {'A': 3.697300e-04, 'n': 5.139, 'Ea': 28.848}, # H(5) + C2HF3(700) <=> HF(684) + C2HF2(717)
    {'A': 4.337330e+00, 'n': 3.866, 'Ea': 42.632}, # CHFO(692) + CF2CH(718) <=> HCO(14) + C2HF3(700)
    {'A': 2.377800e-06, 'n': 5.814, 'Ea': 31.281}, # H(5) + CHF3(687) <=> HF(684) + CHF2(727)
    {'A': 1.723350e-02, 'n': 4.187, 'Ea': 40.845}, # CHF2(727) + CHFO(692) <=> CHF3(687) + HCO(14)
    {'A': 5.729900e+00, 'n': 3.761, 'Ea': 40.524}, # CHF2(727) + C2HF3(700) <=> CHF3(687) + CF2CH(718)
    {'A': 8.360160e+00, 'n': 3.807, 'Ea': 41.954} # CHF3(687) + C2HF2(717) <=> CHF2(727) + C2HF3(700)
]

    
for old_dic, new_dic in zip(old, new):
    A = old_dic['A']
    n = old_dic['n']
    Ea = old_dic['Ea']
    old_rate = get_rate_coefficient(A, n, Ea)
    
    A = new_dic['A']
    n = new_dic['n']
    Ea = new_dic['Ea']
    new_rate = get_rate_coefficient(A, n, Ea)    
    
    print(f'{old_rate:e}', f'{new_rate:e}')





2.056471e+08 1.843728e+08
5.256200e+08 2.527304e+08
1.708269e+08 9.146354e+07
1.396816e+06 2.931501e+05
6.090921e+07 7.118397e+07
1.079331e+05 2.087888e+05
1.839870e+06 1.466456e+06
2.240517e+06 7.504312e+04


In [4]:
#load in the database
database = RMGDatabase()
database.load(
            path = settings['database.directory'],
            thermo_libraries = [],
            transport_libraries = [],
            reaction_libraries = [],
            seed_mechanisms = [],#['BurkeH2O2inN2','ERC-FoundationFuelv0.9'],
            kinetics_families = 'all',
            kinetics_depositories = ['training'],
            #frequenciesLibraries = self.statmechLibraries,
            depository = False, # Don't bother loading the depository information, as we don't use it
        )

In [9]:
H='[H]'
CHFCF2='FC=C(F)F'
HF='F'
CF2CH='[CH]=C(F)F' 


reactant_Molecule = [Molecule(smiles=H), Molecule(smiles=CHFCF2)]
product_Molecule = [Molecule(smiles=HF), Molecule(smiles=CF2CH),] 


reactant_Species = [Species(smiles=H), Species(smiles=CHFCF2)]
product_Species = [Species(smiles=HF), Species(smiles=CF2CH),] 



template_reaction = database.kinetics.families['F_Abstraction'].generate_reactions(reactants=reactant_Molecule, products=product_Molecule) #just choose the first
#template_reaction = database.kinetics.families['F_Abstraction'].generate_reactions(reactants=reactant_Species, products=product_Species) #just choose the first


In [5]:
[([reac_1, reac_2], [prod_1, prod_2])] = [(['C=CF', '[H]'], ['[CH]=C', 'F'])]



reactant_Molecule = [Molecule(smiles=reac_1), Molecule(smiles=reac_2)]
product_Molecule = [Molecule(smiles=prod_1), Molecule(smiles=prod_2),] 




template_reaction = database.kinetics.families['F_Abstraction'].generate_reactions(reactants=reactant_Molecule, products=product_Molecule) #just choose the first


In [6]:
template_reaction

[TemplateReaction(reactants=[Molecule(smiles="C=CF"), Molecule(smiles="[H]")], products=[Molecule(smiles="F"), Molecule(smiles="[CH]=C")], pairs=[[Molecule(smiles="C=CF"), Molecule(smiles="[CH]=C")], [Molecule(smiles="[H]"), Molecule(smiles="F")]], family='F_Abstraction', template=['Root_N-1R->O_N-3R->O_1BrCClFHINPSSi->C_N-3CClFH->F_N-3CClH->C_Ext-1C-R_N-4R!H->F'])]

In [7]:
y = template_reaction[0]

In [8]:
family = database.kinetics.families['F_Abstraction']

In [9]:
x =family.get_kinetics(y, y.template)

In [11]:
rate_rule = [kinetic for kinetic in x if 'rate rules' in kinetic]

In [11]:
x

[[ArrheniusBM(A=(1.0547e-11,'m^3/(mol*s)'), n=5.45748, w0=(525000,'J/mol'), E0=(136285,'J/mol'), Tmin=(300,'K'), Tmax=(2000,'K'), uncertainty=RateUncertainty(mu=0.0028779226243416762, var=0.9461506762015217, Tref=1000.0, N=6, data_mean=0.0, correlation='Root_N-1R->O_N-3R->O_1BrCClFHINPSSi->C_N-3CClFH->F_N-3CClH->C_Ext-1C-R_N-4R!H->F',), comment="""Estimated from node Root_N-1R->O_N-3R->O_1BrCClFHINPSSi->C_N-3CClFH->F_N-3CClH->C_Ext-1C-R_N-4R!H->F"""),
  'rate rules',
  <Entry index=122 label="Root_N-1R->O_N-3R->O_1BrCClFHINPSSi->C_N-3CClFH->F_N-3CClH->C_Ext-1C-R_N-4R!H->F">,
  True],
 [ArrheniusBM(A=(2.64503e+22,'m^3/(mol*s)'), n=-4.37866, w0=(409075,'J/mol'), E0=(226619,'J/mol'), Tmin=(300,'K'), Tmax=(2000,'K'), uncertainty=RateUncertainty(mu=0.7907502389793126, var=30.171198606741513, Tref=1000.0, N=234, data_mean=0.0, correlation='Root',), comment="""Estimated from node RootMatched node Root_N-1R->O_N-3R->O_1BrCClFHINPSSi->C_N-3CClFH->F_N-3CClH->C_Ext-1C-R_N-4R!H->F >> Root_N-1R->O_

In [12]:
y.kinetics = x[0][0]

In [14]:
y.get_rate_coefficient(1400,101325)*(100**3)

102981.81100654523

In [17]:
x

[[Arrhenius(A=(6.87,'cm^3/(mol*s)'), n=3.88, Ea=(124.4,'kJ/mol'), T0=(1,'K'), Tmin=(300,'K'), Tmax=(2000,'K'), comment="""Matched reaction 103 CHFO + H <=> CHO + HF in F_Abstraction/training
  This reaction matched rate rule [Root_N-1R->O_N-3R->O_1BrCClFHINPSSi->C_N-3CClFH->F_N-3CClH->C_Ext-1C-R_N-4R!H->F_Sp-4BrBrCCClClIINNOOPPSSSiSi=1C]
  family: F_Abstraction"""),
  <KineticsDepository "F_Abstraction/training">,
  <Entry index=103 label="CHFO + H <=> CHO + HF">,
  True],
 [ArrheniusBM(A=(2.54565e-18,'m^3/(mol*s)'), n=7.5188, w0=(525000,'J/mol'), E0=(132460,'J/mol'), Tmin=(300,'K'), Tmax=(2000,'K'), uncertainty=RateUncertainty(mu=0.6061594411776685, var=1.3263051219430193, Tref=1000.0, N=2, data_mean=0.0, correlation='Root_N-1R->O_N-3R->O_1BrCClFHINPSSi->C_N-3CClFH->F_N-3CClH->C_Ext-1C-R_N-4R!H->F_Sp-4BrBrCCClClIINNOOPPSSSiSi=1C',), comment="""Estimated from node Root_N-1R->O_N-3R->O_1BrCClFHINPSSi->C_N-3CClFH->F_N-3CClH->C_Ext-1C-R_N-4R!H->F_Sp-4BrBrCCClClIINNOOPPSSSiSi=1C"""),
  'ra

In [29]:
family.get_kinetics?

Signature:
family.get_kinetics(
    reaction,
    template_labels,
    degeneracy=1,
    estimator='',
    return_all_kinetics=True,
)
Docstring:
Return the kinetics for the given `reaction` by searching the various
depositories as well as generating a result using the user-specified `estimator`
of either 'group additivity' or 'rate rules'.  Unlike
the regular :meth:`get_kinetics()` method, this returns a list of
results, with each result comprising of

1. the kinetics
2. the source - this will be `None` if from a template estimate
3. the entry  - this will be `None` if from a template estimate
4. is_forward a boolean denoting whether the matched entry is in the same
   direction as the inputted reaction. This will always be True if using
   rates rules or group additivity. This can be `True` or `False` if using
   a depository

If return_all_kinetics==False, only the first (best?) matching kinetics is returned.
File:      ~/Code/RMG-Py/rmgpy/data/kinetics/family.py
Type:      method


In [ ]:
y.get_rate_coefficient(1400,101325)

In [17]:
dir(y)

['SurfaceArrhenius',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__pyx_vtable__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_degeneracy',
 '_repr_png_',
 'allow_max_rate_violation',
 'allow_pdep_route',
 'calculate_coll_limit',
 'calculate_microcanonical_rate_coefficient',
 'calculate_tst_rate_coefficient',
 'calculate_tst_rate_coefficients',
 'can_tst',
 'check_collision_limit_violation',
 'comment',
 'copy',
 'degeneracy',
 'draw',
 'duplicate',
 'elementary_high_p',
 'ensure_species',
 'estimator',
 'family',
 'fix_barrier_height',
 'fix_diffusion_limited_a_factor',
 'generate_3d_ts',
 'generate_high_p_limit_kinetics',
 'generate_pairs',
 'generate_reverse_rate_coefficient',
 'get_enthalpies_of_react

In [ ]:
y.network_kinetics

In [ ]:
manual comparison     Ea in kcal/mol
new                         
old 

equation: H(5) + CHFCF2(54) <=> HF(38) + CF2CH(72)  new
rate-constant: {A: 1.0547e-05, b: 5.457, Ea: 25.149} new
reaction('H(5) + C2HF3(700) <=> HF(684) + CF2CH(718)', [1.575380e-09, 6.622, 23.817]) old
    
H(5)+CHFO(49)<=>HF(36)+HCO(14)                      6.870000e+00 3.880     29.732  NEW
reaction('H(5) + CHFO(692) <=> HF(684) + HCO(14)', [3.318290e-02, 4.482, 24.991]) old    


- equation: H(5) + CHFCF2(54) <=> HF(38) + CHFCF[Z](71)  # Reaction 455 new 
  rate-constant: {A: 3.6973e-04, b: 5.139, Ea: 30.586} new
reaction('H(5) + C2HF3(700) <=> HF(684) + C2HF2(717)', [3.697300e-04, 5.139, 28.848]) old
    
    
reaction('CHFO(49) + CF2CH(740) <=> HCO(14) + CHFCF2(80)', [5.861460e-04, 4.798, 40.973])new
reaction('CHFO(692) + CF2CH(718) <=> HCO(14) + C2HF3(700)', [4.337330e+00, 3.866, 42.632])old

    
reaction('C2HF2(717) + C2HF3(700) <=> CF2CH(718) + C2HF3(700)', [5.729900e+00, 3.761, 40.477]) old 
cant find new rate

reaction('H(5) + CHF3(717) <=> HF(36) + CHF2(25)', [1.920000e+00, 4.04, 32.935]) new
reaction('H(5) + CHF3(687) <=> HF(684) + CHF2(727)', [2.377800e-06, 5.814, 31.281])old


reaction('CHF2(25) + CHFO(49) <=> CHF3(717) + HCO(14)', [3.630000e+01, 3.37, 43.834]) new
reaction('CHF2(727) + CHFO(692) <=> CHF3(687) + HCO(14)', [1.723350e-02, 4.187, 40.845]) old

reaction('CHF2(25) + CHFCF2(80) <=> CHF3(717) + CF2CH(740)', [1.527230e-01, 4.186, 39.636]) new
reaction('CHF2(727) + C2HF3(700) <=> CHF3(687) + CF2CH(718)', [5.729900e+00, 3.761, 40.524]) old

reaction('CHF3(717) + CHFCF[Z](739) <=> CHF2(25) + CHFCF2(80)', [8.880000e+01, 3.23, 46.348]) new
reaction('CHF3(687) + C2HF2(717) <=> CHF2(727) + C2HF3(700)', [8.360160e+00, 3.807, 41.954])old

CHF3(42)+C2H(22)<=>CHF2(82)+C2HF(58)                7.441770e+01 3.451     42.307   new BUT BACKWARD
reaction('CHF2(727) + C2HF(703) <=> CHF3(687) + C2H(28)', [2.843880e-01, 3.89, 41.58]) old






In [ ]:
manual comparison     Ea in kcal/mol
new                         
old 

equation: H(5) + CHFCF2(54) <=> HF(38) + CF2CH(72)  {A: 1.0547e-05, b: 5.457, Ea: 25.149} new good
reaction('H(5) + C2HF3(700) <=> HF(684) + CF2CH(718)', [1.575380e-09, 6.622, 23.817]) old
    
    
H(5)+CHFO(49)<=>HF(36)+HCO(14)                      6.870000e+00 3.880     29.732   new good
- equation: H(5) + CHFO(46) <=> HF(38) + HCO(14)  # Reaction 452 old 
  rate-constant: {A: 6.87, b: 3.88, Ea: 29.732} old

- equation: H(5) + CHFCF2(54) <=> HF(38) + CHFCF[Z](71) rate-constant: {A: 3.6973e-04, b: 5.139, Ea: 30.586} new good
reaction('H(5) + C2HF3(700) <=> HF(684) + C2HF2(717)', [3.697300e-04, 5.139, 28.848]) old
    
    
reaction('CHFO(49) + CF2CH(740) <=> HCO(14) + CHFCF2(80)', {A: 5.861460e-04, b: 4.798, Ea: 40.973}   new good
reaction('CHFO(692) + CF2CH(718) <=> HCO(14) + C2HF3(700)', [4.337330e+00, 3.866, 42.632])old

    
reaction('C2HF2(717) + C2HF3(700) <=> CF2CH(718) + C2HF3(700)', [5.729900e+00, 3.761, 40.477]) old 
cant find new rate

H(5)+CHF3(717)<=>HF(36)+CHF2(25)                    1.920000e+00 4.040     32.935  new good
reaction('H(5) + CHF3(687) <=> HF(684) + CHF2(727)', [2.377800e-06, 5.814, 31.281])old


CHF2(25)+CHFO(49)<=>CHF3(717)+HCO(14)               3.630000e+01 3.370     43.834  new good
reaction('CHF2(727) + CHFO(692) <=> CHF3(687) + HCO(14)', [1.723350e-02, 4.187, 40.845]) old

reaction('CHF2(25) + CHFCF2(80) <=> CHF3(717) + CF2CH(740)', [1.527230e-01, 4.186, 39.636]) new
reaction('CHF2(727) + C2HF3(700) <=> CHF3(687) + CF2CH(718)', [5.729900e+00, 3.761, 40.524]) old

reaction('CHF3(717) + CHFCF[Z](739) <=> CHF2(25) + CHFCF2(80)', [8.880000e+01, 3.23, 46.348]) new
reaction('CHF3(687) + C2HF2(717) <=> CHF2(727) + C2HF3(700)', [8.360160e+00, 3.807, 41.954])old

CHF3(42)+C2H(22)<=>CHF2(82)+C2HF(58)                7.441770e+01 3.451     42.307   new BUT BACKWARD
reaction('CHF2(727) + C2HF(703) <=> CHF3(687) + C2H(28)', [2.843880e-01, 3.89, 41.58]) old






In [ ]:
    cpdef double get_rate_coefficient(self, double T, double P=0.0) except -1:
        """
        Return the rate coefficient in the appropriate combination of m^3,
        mol, and s at temperature `T` in K.
        """
        cdef double A, n, Ea, T0
        A = self._A.value_si
        n = self._n.value_si
        Ea = self._Ea.value_si
        T0 = self._T0.value_si
        return A * (T / T0) ** n * exp(-Ea / (constants.R * T))